In [ ]:
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

print(" JSON Schema & Logit Bias Explorer Ready!")

In [ ]:

schema = {
    "type": "object",
    "properties": {
        "title": {"type": "string"},
        "rating": {"type": "integer", "minimum": 1, "maximum": 5},
        "review": {"type": "string"}
    },
    "required": ["title", "rating", "review"]
}

prompt = "Generate a book review for '1984' by George Orwell in JSON format."

print(" JSON Schema & Temperature Experiment")
print(f"Prompt: '{prompt}'\n")


temperatures = [0.0, 0.7, 1.2]

for temp in temperatures:
    print(f" Temperature: {temp}")
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"Return valid JSON: {json.dumps(schema)}"},
            {"role": "user", "content": prompt}
        ],
        temperature=temp,
        response_format={"type": "json_object"},
        max_tokens=200
    )
    
    try:
        json_result = json.loads(response.choices[0].message.content)
        print(" Valid JSON:")
        print(f"  Title: {json_result.get('title', 'N/A')}")
        print(f"  Rating: {json_result.get('rating', 'N/A')}")
        print(f"  Review: {json_result.get('review', 'N/A')[:50]}...")
    except json.JSONDecodeError:
        print(" Invalid JSON generated")
    print()

In [ ]:

print(" Logit Bias Experiment")


token_map = {
    "excellent": 12481,
    "amazing": 21088, 
    "terrible": 14935,
    "boring": 33510
}


bias_tests = [
    ({}, "No bias"),
    ({str(token_map["excellent"]): 2.0}, "Bias toward 'excellent'"),
    ({str(token_map["terrible"]): -5.0}, "Suppress 'terrible'")
]

for bias_dict, desc in bias_tests:
    print(f"\n {desc}:")
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "Write a short review of 1984 by George Orwell."}],
        temperature=0.8,
        logit_bias=bias_dict,
        max_tokens=100
    )
    
    print(f"Result: {response.choices[0].message.content}")
    print()